# Hardcoded Gesture Recognition with MediaPipe and Webcam

In [1]:
import cv2
import time
import mediapipe as mp
import os

In [2]:
from utils.basic_gesture_control import recognize_gesture

In [3]:
VIDEO_FOLDER = "test_videos"

In [4]:
mp_hands = mp.solutions.hands
mp_draw = mp.solutions.drawing_utils

In [5]:
os.makedirs(VIDEO_FOLDER, exist_ok=True)

In [9]:
timestamp = time.strftime("%Y%m%d_%H%M%S")
output_path = os.path.join(VIDEO_FOLDER, f"tello_gesture_hardcoded_{timestamp}.avi")

cap = cv2.VideoCapture(0)
width, height = 640, 480
cap.set(cv2.CAP_PROP_FRAME_WIDTH, width)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, height)

out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'XVID'), 20.0, (width, height))

with mp_hands.Hands(
    max_num_hands=1,
    min_detection_confidence=0.7,
    min_tracking_confidence=0.5) as hands:

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Flip horizontal pour effet miroir
        frame = cv2.flip(frame, 1)
        image_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        
        results = hands.process(image_rgb)

        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:
                # Affichage des points
                mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                
                gesture = recognize_gesture(hand_landmarks.landmark)
                
                # Affichage sur la frame
                cv2.putText(frame, f'Gesture: {gesture}', (10, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
                
        out.write(frame)
        cv2.imshow("Hand Gesture Recognition", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

hands.close()
cap.release()
out.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 